In [5]:
import matplotlib.pyplot as plt

import matplotlib.patches as patches
import numpy as np
import group_glm as gm
from glm_utils import *
from scipy.stats import linregress
import pprint
import allensdk.brain_observatory.stimulus_info as stim_info
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import tensorflow as tf
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
%matplotlib inline

boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
#first we get the data

regions = ['VISal']
lines =  ['Cux2-CreERT2']#, 'Rbp4-Cre', 'Rorb-IRES2-Cre'] 

data_set = download_data(regions, lines, [stim_info.NATURAL_SCENES])

In [27]:
all_tensors = {}
ra_variance = {}
dffs = {}
data_dict = {}

var_thresh = .1
for key in data_set.keys():
    dff, images, stim_table = get_data(data_set[key], stim_info.NATURAL_SCENES)
    data_tensor, trialCount = arrange_ns_data_trialTensor(dff, stim_table)
    all_tensors[key] = data_tensor
    
    ra_variance[key] = get_explainable_variance(all_tensors[key])
    
    exp_var = ra_variance[key]
    exp_ind = exp_var > var_thresh
    
    pruned_dff_traces = dff[exp_ind]
    
    print "neurons with >", 100 * var_thresh, "% explainable variance for dataset: ", key, " : ", sum(exp_ind)
    
    r_images = pca_features(images)
    
    data, stim_array = arrange_data_glm(pruned_dff_traces, r_images, stim_table)
    
    dxcm, dtime = data_set[key].get_running_speed()
    rs = arrange_rs_glm(dxcm, stim_table)
    
    data_dict[key] = (data, stim_array, rs)
    

neurons with > 10.0 % explainable variance for dataset:  504625475  :  33
neurons with > 10.0 % explainable variance for dataset:  503820068  :  22
neurons with > 10.0 % explainable variance for dataset:  511976329  :  10
neurons with > 10.0 % explainable variance for dataset:  506156402  :  11
neurons with > 10.0 % explainable variance for dataset:  507691380  :  6
neurons with > 10.0 % explainable variance for dataset:  512149367  :  14
neurons with > 10.0 % explainable variance for dataset:  502382906  :  45
neurons with > 10.0 % explainable variance for dataset:  501889084  :  39
neurons with > 10.0 % explainable variance for dataset:  501929146  :  41


In [28]:
for key in data_dict.keys():
    dxcm, dxtime = data_set[key].get_running_speed()
    
    data, stim_array, rs = data_dict[key]
    print key
    
    print np.isnan(dxcm)

504625475
[ True  True  True ...,  True  True  True]
503820068
[ True  True  True ...,  True  True  True]
511976329
[ True  True  True ...,  True  True  True]
506156402
[ True  True  True ...,  True  True  True]
507691380
[ True  True  True ...,  True  True  True]
512149367
[ True  True  True ...,  True  True  True]
502382906
[ True  True  True ...,  True  True  True]
501889084
[ True  True  True ...,  True  True  True]
501929146
[ True  True  True ...,  True  True  True]


In [29]:
data_set[key].get_running_speed()

(array([ nan,  nan,  nan, ...,  nan,  nan,  nan]),
 array([   17.23731,    17.27056,    17.30381, ...,  3802.9257 ,
         3802.95895,  3802.9922 ]))

In [30]:
ds = data_set[key]

In [31]:
dxcm, dxtime = ds.get_running_speed()